In [1]:
from google.colab import userdata
!pip install -qU "langchain-community>=0.2.11" tavily-python
key = userdata.get('TAVILY_API_KEY')
import subprocess
import os

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = key
def check_and_install(package_name):
    try:
        subprocess.check_output(['pip', 'show', package_name])
    except subprocess.CalledProcessError:
        print(f"{package_name} not found, installing...")
        subprocess.check_output(['pip', 'install', package_name])
    else:
        print(f"{package_name} already installed.")

if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')

check_and_install('langchain_community')
check_and_install('langchain')
check_and_install('openai')
check_and_install('python-dotenv')
check_and_install('tiktoken')
check_and_install('langchain_openai')
check_and_install('google-colab')
check_and_install('langgraph')
check_and_install('chromadb')
check_and_install('langchain_experimental')
!pip install --upgrade langchain
!pip install langgraph
!pip install chromadb

oai_key = userdata.get('OPENAI_API_KEY')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.7 MB/s eta 0:00:00
Running on CoLab
langchain_community already installed.
langchain already installed.
openai already installed.
python-dotenv already installed.
tiktoken already installed.
langchain_openai not found, installing...
google-colab already installed.
langgraph not found, installing...
chromadb not found, installing...
langchain_experimental not found, installing...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.8 MB/s eta 0:00:00
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.16
    Uninstalling langchain-0.3.16:
      Successfully uninstalled langchain-0.3.16


In [3]:
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI



prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an essay assistant tasked with writing excellent 5-paragraph essays."
            " Generate the best essay possible for the user's request."
            " If the user provides critique, respond with a revised version of your previous attempts.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
llm = ChatOpenAI(model="gpt-4o-mini", api_key=oai_key)

generate = prompt | llm

In [4]:
essay = ""
request = HumanMessage(
    content="Write an essay on why the little prince is relevant in modern childhood"
)
for chunk in generate.stream({"messages": [request]}):
    print(chunk.content, end="")
    essay += chunk.content

**Title: The Timeless Relevance of 'The Little Prince' in Modern Childhood**

**Introduction**

Antoine de Saint-Exupéry's novella, 'The Little Prince,' has captivated readers of all ages since its publication in 1943. This enchanting tale, while ostensibly a children's story, transcends generational boundaries and continues to resonate profoundly with modern childhood experiences. In an age dominated by technology, rapid changes, and societal challenges, the themes encompassed within 'The Little Prince' hold critical relevance for today's youth. The story's emphasis on imagination, emotional intelligence, and the importance of human connections provides essential lessons that help guide children toward developing a compassionate and meaningful worldview.

**Imagination and Creativity**

One of the most significant aspects of 'The Little Prince' is its celebration of imagination and creativity. In a world increasingly influenced by screens and digital engagement, children often find th

In [5]:
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a teacher grading an essay submission. Generate critique and recommendations for the user's submission."
            " Provide detailed recommendations, including requests for length, depth, style, etc.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
reflect = reflection_prompt | llm

In [6]:
reflection = ""
for chunk in reflect.stream({"messages": [request, HumanMessage(content=essay)]}):
    print(chunk.content, end="")
    reflection += chunk.content

**Critique of the Essay Submission: "The Timeless Relevance of 'The Little Prince' in Modern Childhood"**

Your essay presents a thoughtful and well-structured exploration of why "The Little Prince" remains relevant in the context of modern childhood. You effectively touch upon key themes such as imagination, emotional intelligence, and the formation of human connections. However, while you establish a strong argument, there are areas where you could enhance the depth and clarity of your analysis.

### General Recommendations:

1. **Depth of Analysis:**
   - While you mention significant themes, consider delving deeper into specific examples from the text to illustrate your points. For instance, when discussing imagination, you could highlight particular adventures of the Little Prince and their relevance to encouraging creativity in children today. Incorporate quotations from the text to support your claims and provide further insight.

2. **Length and Detail:**
   - Expanding on each

In [7]:
for chunk in generate.stream(
    {"messages": [request, AIMessage(content=essay), HumanMessage(content=reflection)]}
):
    print(chunk.content, end="")

**Title: The Timeless Relevance of 'The Little Prince' in Modern Childhood**

**Introduction**

Antoine de Saint-Exupéry's novella, 'The Little Prince,' has enchanted readers of all ages since its publication in 1943. While it is often categorized as a children’s story, its insights resonate significantly with contemporary children, navigating a world marked by rapid technological advances and societal complexities. This essay argues that 'The Little Prince' remains profoundly relevant in modern childhood due to its celebration of imagination, its promotion of emotional intelligence, and its emphasis on the importance of human connections. By unpacking these themes with specific examples from the text and relating them to today's challenges, we can better appreciate the enduring lessons of this beloved tale.

**Imagination and Creativity**

At the heart of 'The Little Prince' lies a vibrant celebration of imagination and creativity. The Little Prince's whimsical adventures across vario

In [8]:
from typing import Annotated, List, Sequence
from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from typing_extensions import TypedDict


class State(TypedDict):
    messages: Annotated[list, add_messages]


async def generation_node(state: State) -> State:
    return {"messages": [await generate.ainvoke(state["messages"])]}


async def reflection_node(state: State) -> State:
    # Other messages we need to adjust
    cls_map = {"ai": HumanMessage, "human": AIMessage}
    # First message is the original user request. We hold it the same for all nodes
    translated = [state["messages"][0]] + [
        cls_map[msg.type](content=msg.content) for msg in state["messages"][1:]
    ]
    res = await reflect.ainvoke(translated)
    # We treat the output of this as human feedback for the generator
    return {"messages": [HumanMessage(content=res.content)]}


builder = StateGraph(State)
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.add_edge(START, "generate")


def should_continue(state: State):
    if len(state["messages"]) > 6:
        # End after 3 iterations
        return END
    return "reflect"


builder.add_conditional_edges("generate", should_continue)
builder.add_edge("reflect", "generate")
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

In [9]:
config = {"configurable": {"thread_id": "1"}}

In [10]:
async for event in graph.astream(
    {
        "messages": [
            HumanMessage(
                content="Generate an essay on the topicality of The Little Prince and its message in modern life"
            )
        ],
    },
    config,
):
    print(event)
    print("---")

{'generate': {'messages': [AIMessage(content='**The Timeless Relevance of "The Little Prince" in Modern Life**\n\nAntoine de Saint-Exupéry’s "The Little Prince," first published in 1943, is a novella that, despite its status as a children\'s book, resonates profoundly with adults and children alike. Its themes of love, friendship, and the quest for meaning transcend the boundaries of culture and time. In an age marked by rapid technological advancement and shifting social dynamics, "The Little Prince" remains remarkably relevant. This essay explores the guiding messages of the book and their significance in addressing modern life\'s complexities.\n\nOne of the central themes of "The Little Prince" is the importance of seeing beyond the superficial. The protagonist, a young prince from a distant asteroid, travels to different planets and encounters various inhabitants, each representing distinct societal flaws. For example, the king, the businessman, and the geographer are consumed by p